In [ ]:
import pandas as pd
import json

In [ ]:
import pandas as pd
import re

sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
groups = []
with open("CS_Word_20240405_22.25.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if matched == True and last_line[-1] != "。":
            sentence["content"] += line
            last_line = line
            continue

        if sentence_content_pattern.match(line.strip()):
            matched = True
            sentence = {"content": sentence_content_pattern.sub(r"\1 ", line)}
        else:
            if matched:
                sentence["hiragana"] = ""
                sentence["meaning"] = line
                if sentence["meaning"][1] == " ":
                    sentence["meaning"] = sentence["meaning"][2:]

                if sentence["meaning"][0] == "囉":
                    sentence["meaning"] = sentence["meaning"][1:]

                if sentence["content"][0] == "①" and groups:
                    example_sentences.append(groups)
                    groups = []

                groups.append(sentence)
            matched = False
        last_line = line

if groups:
    example_sentences.append(groups)

print(example_sentences[1])
with open("example_sentences.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(example_sentences, ensure_ascii=False, indent=4))
    # pd.DataFrame(example_sentences, dtype=str).fillna("").to_json(fp, orient="records", force_ascii=False, indent=4)

In [ ]:
import json
import pandas as pd

df = pd.read_json("grammar_pd.json", dtype=str)
with open("grammar.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())
    df = pd.DataFrame(data)
df.head()
detail = df.loc[0].to_dict()
print(type(detail["example"]))
print(detail["example"])

In [2]:
from grammar_enumeration import grammars, df_grammars

In [21]:
df_grammars.shape

(238, 10)

In [30]:
df_grammars.hiragana.str.replace(r"[（）()\s()]", "").str.contains(r'なしに')[23]

True

In [3]:
keyword = "なくして(は)"
df = df_grammars
df[df.content.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.hiragana.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.chinese_meaning.str.contains(keyword) | (df.source == keyword)]

c:\python38\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,id,content,hiragana,meaning,usage,example,remark,source,japanese_meaning,chinese_meaning
182,182,〜なしに(は)~なしではなくして(は),,"⇒ もし〜がなかったら、 あることが成立しない。\n如果没有······的活,某事就法迸行。",名・動 辞書形+ こと +なしに(は)なしではなくして(は),"[{'content': '① 資金を確保することなしにはどんな計画も実行できない。', '...","表述说话人的某种判断评价,认为要使某事得以实施,......是必要条件。前接表示说话人认为绝...",S1N109,⇒ もし〜がなかったら、 あることが成立しない。,"如果没有······的活,某事就法迸行。"


In [ ]:
import text_to_audio

In [ ]:
text_to_audio.generate_audio([
    (None, "このマラソン大会には、性別、年齢を問わず"),
], None)

In [ ]:
grammar_content_pattern = re.compile(r"^\d? ?[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")

grammar_content_pattern.findall("~ 際 (に)")

In [ ]:
content = "ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"
content = re.sub("\s+", " ", content)
print(content)

question_pattern = re.compile(r"[（(](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[)）]")
print(question_pattern.findall(content))

In [ ]:
re.findall(r"(?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+", "a だけ b に限り")

In [ ]:
question_pattern = re.compile(r"[(（](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[）)]*\?")
print(question_pattern.findall("こちらの会議室をご利用になる際は、 受付で必要事項をご記入ください。"))
print(question_pattern.findall("ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"))

In [ ]:
import pandas as pd
import re

grammar_content_pattern = re.compile(r"^\d [~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")
sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
with open("CS_Word_20240403_23.41.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if grammar_content_pattern.match(line):
            print(line)